In [1]:
import random
import json
import urllib.parse

In [25]:
#No-sql injection commands

queries = [
    ('{"user": "{}", "password": "{}"}', 0),
    ('{"username": {"$regex": "^{}"}}', 1),
    ('{"username": {"$ne": "{}"}}', 1),
    ('{"username": {"$gt": "{}"}}', 1),
    ('{"user": "hacker", "password": {"$ne": ""}}', 1),
    ('{"user": req.query.user, "password": req.query.password}', 0),
    ('{"user": "hacker", "password": {"&ne": ""}}', 1),
    ('{"user": req.params[\'user\'], "password": req.params[\'password\']}', 0),
    ('{"user": request.GET[\'user\'], "password": request.GET[\'password\']}', 0),
    ('{"user": req.query.user_id, "password": req.query.user_pass}',0),
    ('{"user": req.params[\'user_id\'], "password": req.params[\'user_pass\']}',0),
    ('db.users.find({"user": "user123", "email": "user123@example.com"})',0),
    ('db.users.find({"user": request.GET[\'user\'], "password": request.GET[\'password\']})', 0),
    ('db[\'users\'].find({:user => req.params[\'user\'], :password => req.params[\'password\']})', 0),
    ('db.collection(\'users\').find({"user": "hacker", "password": {"&ne": ""}})', 1),
    ('}); db.dropDatabase(); db.insert({username: ‘dummy ’, password: ‘dummy’', 1),
    ('{"username": {"$eq": "admin"}, "password": {"$regex": "^md" }}', 0), 
    ('login[$nin][]=admin&login[$nin][]=test&pass[$ne]=toto', 1), 
    ("login%255B%2524gt%255D%3Dadmin%26login%255B%2524lt%255D%3Dtest%26pass%255B%2524ne%255D%3D1", 1), 
    ('{"type": "user","name": "oops", "roles": ["_admin"], "roles": [], "password":"apple"}', 1),
    ('true, $where: \'1 == 1\'', 1),
    ('{"$where": "this.username === \'{}\'"}', 1),
    ('{"$or": [{"username": "{}"}, {"isAdmin": true}]}', 1),
    ('{ "$or": [ {}, { "a": "a' , 1),
    ("' } ], $comment:'successful MongoDB injection'", 1),
    ('{"user": "alice", "password": "123456"}', 0),
    ('{"username": {"$regex": "^test"}}', 0),
    ('{"username": {"$ne": "testuser"}}', 0),
    ('{"username": {"$gt": "guest"}}', 0),
    ('{"user": "john", "password": "qwerty"}', 0),
    ('{"user": req.query.name, "password": req.query.pass}', 0),
    ('{"user": req.params[\'name\'], "password": req.params[\'pass\']}', 0),
    ('{"user": request.GET[\'name\'], "password": request.GET[\'pass\']}', 0),
    ('db.users.find({"user": "john", "age": {"$gt": 25}})', 0),
    ('db.users.find({"user": "user", "age": {"$gt": 30}})',0),
    ('db.users.find({"user": request.GET[\'name\'], "password": request.GET[\'pass\']})', 0),
    ('db[\'users\'].find({:user => req.params[\'name\'], :password => req.params[\'pass\']})', 0),
    ("' && this.password.match(/.*/)//+%00", 1),
    ("' && this.passwordzz.match(/.*/)//+%00", 1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("0;return true",1),
    ("|| 1==1",1),
    ("db.injection.insert({success:1});",1),
    ("|| sleep(5)",1),
    ('%27%3B%20return%20%27%27%20==%20%27', 1),
    ("login%5B%24gt%5D=admin&login%5B%24lt%5D=test&pass%5B%24ne%5D=1",1),
    ('%27%20&&%20this.password.match%28%2F.%2A%2F%29%2F%2F%2B%2500', 1),
    ("db.injection.insert({success:1});",1),
    ("|| sleep(5)",1),
    ("|| 1==1",1),

    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("0;return true",1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("|| this.password.match(/.*/)//+%00", 1),
    ("' && this.password.match(/.*/)//+%00", 1),
    ("' && this.passwordzz.match(/.*/)//+%00", 1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("true, $where: '1 == 1'",1),
    ("$where: '1 == 1'",1),
    ("db.collection('users').find({:user => req.params['user'], :password => req.params['password']})",0),
    ("this.hidden == false && this.category == ''; return '' == ''",0),
    (r"}); db.dropDatabase();", 1), #r is for raw string    
    ('{"category": "electronics"}', 0),
    ('{"category": "electronics", "price": {"$gt": 200}}', 0),
    ('{"category": "electronics", "price": {"$gt": 200}, "name": {"$regex": "^S"}}', 0),
    ('{}', 0),
    ('{"selector": {"department": "engineering"}}', 0),
    ('{"_id": "001", "name": "John Doe", "department": "HR", "salary": 50000}', 0),
    ("SELECT * FROM beer-sample WHERE city='' AND '1'='0' UNION SELECT * FROM system:keyspaces WHERE '1'='1'", 1),
    ("PartitionKey eq 'Customers' and RowKey eq '12345'", 0),
    ("PartitionKey eq 'Employees' and RowKey eq '44455' and Password eq 'password'", 0),
    ("PartitionKey eq 'Children' and RowKey eq '1253' and  Email eq 'email'", 0),
    ("PartitionKey eq 'Patients' and RowKey eq '15334' and Phone eq 'phone'", 0),
    ("0;var date=new Date(); do{curDate = new Date();}while(curDate-date<10000)",1),
    ('{"user": "{}", "password": "{}"}', 0),
    ('{"username": {"$regex": "^{}"}}', 1),
    ('{"username": {"$ne": "{}"}}', 1),
    ('{"username": {"$gt": "{}"}}', 1),
    ('{"user": "hacker", "password": {"$ne": ""}}', 1),
    ('{"user": req.query.user, "password": req.query.password}', 0),

    ('{"user": "hacker", "password": {"&ne": ""}}', 1),
    ('{"user": req.params[\'user\'], "password": req.params[\'password\']}', 0),
    ('{"user": request.GET[\'user\'], "password": request.GET[\'password\']}', 0),
    ('db.users.find({"user": request.GET[\'user\'], "password": request.GET[\'password\']})', 0),
    ('db[\'users\'].find({:user => req.params[\'user\'], :password => req.params[\'password\']})', 0),
    ('db.collection(\'users\').find({"user": "hacker", "password": {"&ne": ""}})', 1),
    ('}); db.dropDatabase(); db.insert({username: ‘dummy ’, password: ‘dummy’', 1),
    ('{"username": {"$eq": "admin"}, "password": {"$regex": "^md" }}', 0), 
    ('login[$nin][]=admin&login[$nin][]=test&pass[$ne]=toto', 1), 
    ("login%255B%2524gt%255D%3Dadmin%26login%255B%2524lt%255D%3Dtest%26pass%255B%2524ne%255D%3D1", 1), 
    ('{"type": "user","name": "oops", "roles": ["_admin"], "roles": [], "password":"apple"}', 1),
    ('db.users.find({"name": {"$regex": "^j"}})',0),
    ('db.users.find({"bio": {"$exists": true, "$ne": ""}})',0),
    ('true, $where: \'1 == 1\'', 1),
    ('{"$where": "this.username === \'{}\'"}', 1),
    ('{"$or": [{"username": "{}"}, {"isAdmin": true}]}', 1),
    ('{ "$or": [ {}, { "a": "a' , 1),
    ("' } ], $comment:'successful MongoDB injection'", 1),
    ('{"user": "alice", "password": "123456"}', 0),
    ('{"username": {"$regex": "^test"}}', 0),
    ('{"username": {"$ne": "testuser"}}', 0),
    ('{"username": {"$gt": "guest"}}', 0),
    ('{"user": "john", "password": "qwerty"}', 0),
    ('{"user": req.query.name, "password": req.query.pass}', 0),
    ('{"user": req.params[\'name\'], "password": req.params[\'pass\']}', 0),
    ('{"user": request.GET[\'name\'], "password": request.GET[\'pass\']}', 0),
    ('db.users.find({"user": "john", "age": {"$gt": 25}})', 0),
    ('db.users.find({"user": "user", "age": {"$gt": 30}})',0),
    ('db.users.find({"user": request.GET[\'name\'], "password": request.GET[\'pass\']})', 0),
    ('db[\'users\'].find({:user => req.params[\'name\'], :password => req.params[\'pass\']})', 0),
    ("' && this.password.match(/.*/)//+%00", 1),
    ("' && this.passwordzz.match(/.*/)//+%00", 1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("0;return true",1),
    ("|| 1==1",1),
    ("db.injection.insert({success:1});",1),
    ("|| sleep(5)",1),
    ('%27%3B%20return%20%27%27%20==%20%27', 1),
    ("login%5B%24gt%5D=admin&login%5B%24lt%5D=test&pass%5B%24ne%5D=1",1),
    ('%27%20&&%20this.password.match%28%2F.%2A%2F%29%2F%2F%2B%2500', 1),
    ("db.injection.insert({success:1});",1),
    ("|| sleep(5)",1),
    ("|| 1==1",1),
    
    ('db.users.aggregate([{"$group": {"_id": "$phone", "count": {"$sum": 1}}}, {"$match": {"count": {"$gt": 1}}}])', 0),
    ('db.users.find().sort({"status": 1, "age": -1})', 0),
    ('db.users.find({"login_count": {"$gt": 10}, "last_login": {"$gte": ISODate("2023-06-01")}})', 0),
    ('db.users.find({"login_count": {"$gt": 10}, "last_login": {"$gte": ISODate("2023-06-01")}})', 0),
    ('db.users.remove({"user": "user123"})', 0),
    ('db.database.getIndexes()', 0),
    ('db.database.createIndex({"user": 1})', 0),
    ('db.database.stats()', 0),
    ('db.database.drop()', 0),
    ('db.database.insert({$where: "this.user === \\"admin\\" && this.password === \\"password\\"}"})', 1),
    ('db.database.update({"user": "user123"}, {"$set": {$where: "this.password === \\"new_password\\""}})', 1),
    ('db.database.drop({$where: "1 === 1"})', 1),
    ('db.database.stats({$where: "1 === 1"})', 1),
    ('db.database.getIndexes({$where: "1 === 1"})', 1),
    ('db.database.createIndex({$where: "this.user === \\"user123\\"})', 1),
    ('db.database.remove({$where: "this.user === \\"admin123\\""})', 1),

    ('db.users.find({"last_login": {"$gte": ISODate("2023-07-01")}})', 0),
    ('db.users.find({"last_login": {"$lt": ISODate("2023-05-25")}})', 0),
    ('db.users.find().sort({"age": -1})', 0),
    ('db.users.find({"status": "active"})', 0),
    ('db.users.find({"registration_date": {"$gte": ISODate("2023-07-01")}})', 0),
    ('db.users.find({"profile_info": null})', 0),
    ('db.users.find({"user": {"$regex": "^A"}})', 0),
    ('db.users.find({"email": {"$exists": true}})', 0),
    ('db.users.find({"age": {"$gt": 20, "$lt": 40}})', 0),
    ('db.users.find({"role": "admin"})', 0),

    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("0;return true",1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("|| this.password.match(/.*/)//+%00", 1),
    ("' && this.password.match(/.*/)//+%00", 1),
    ("' && this.passwordzz.match(/.*/)//+%00", 1),
    ("';it=new%20Date();do{pt=new%20Date();}while(pt-it<5000);",1),
    ("';sleep(5000);",1),
    ("0;return true;var%20password%20=%20this.password.match(/.*/);",1),
    ("true, $where: '1 == 1'",1),
    ("$where: '1 == 1'",1),
    ("db.collection('users').find({:user => req.params['user'], :password => req.params['password']})",0),
    ("this.hidden == false && this.category == ''; return '' == ''",0), 
    ('{"category": "electronics"}', 0),
    ('{"category": "electronics", "price": {"$gt": 200}}', 0),
    ('{"category": "electronics", "price": {"$gt": 200}, "name": {"$regex": "^S"}}', 0),
    ('{}', 0),
    ('{"selector": {"department": "engineering"}}', 0),
    ('{"_id": "001", "name": "John Doe", "department": "HR", "salary": 50000}', 0),
    ("SELECT * FROM beer-sample WHERE city='' AND '1'='0' UNION SELECT * FROM system:keyspaces WHERE '1'='1'", 1),
    ("PartitionKey eq 'Customers' and RowKey eq '12345'", 0),
    ("PartitionKey eq 'Employees' and RowKey eq '44455' and Password eq 'password'", 0),
    ("PartitionKey eq 'Children' and RowKey eq '1253' and  Email eq 'email'", 0),
    ("PartitionKey eq 'Patients' and RowKey eq '15334' and Phone eq 'phone'", 0),
    ("0;var date=new Date(); do{curDate = new Date();}while(curDate-date<10000)",1),
    ('db.collection.find({$where: "function() {return this.username.indexOf("admin") !== -1}"})',1),
    ('db.collection.find({$or: [{"status": "active"}, {"isAdmin": true}]})',1),
    ('db.collection.find({"email": {"$regex": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$"}})',1),
    ('db.collection.find({$where: "this.password.length >= 12"})',1),

    ('{"user": "{}", "password": "pass"}', 0),  # Variation: fixed password
    ('{"user": "user", "password": "{}"}', 0),  # Variation: fixed username
    ('{"user": "admin", "password": "admin"}', 0),  # Variation: common admin credentials
    ('{"user": "testuser", "password": "testuser"}', 0),  # Variation: common testuser credentials
    ('{"user": "$username", "password": "$password"}', 0),  # Variation: placeholders for username and password
    ('{"id": {}, "value": 1}', 0),  # Variation: different field names and values
    ('{"name": "John", "age": {}}', 0),  # Variation: different field names and values
    ('{"$where": "this.age >= {}"}', 1),  # Variation: $where with a different condition
    ('{"user": "admin", "password": {}}', 1),  # Variation: missing password
    ('{"user": "test", "password": "test"}', 1),  # Variation: common test credentials
    ('{"user": {"$ne": "testuser"}, "password": "pass"}', 1),  # Variation: $ne condition with fixed password
    ('{"username": {"$regex": "^test"}}', 1),  # Variation: $regex condition with a different pattern
    ('{"user": {"$gt": "guest"}, "password": "pass"}', 1),  # Variation: $gt condition with fixed password
    ('{"type": "user", "name": "admin", "roles": ["_admin"], "password": "123456"}', 1),  # Variation: different roles
    ('{ "$or": [ {"username": "admin"}, {"isAdmin": true} ]}', 1),  # Variation: $or condition with isAdmin

    ('{"username": {"$regex": ".*{}.*"}}', 1),  # Variation: $regex with .* surrounding the pattern
    ('{"user": "admin", "password": {"$nin": ["pass", "test"]}}', 1),  # Variation: $nin with multiple values
    ('{"$where": "this.username == \'{}\'"}', 1),  # Variation: $where with == condition
    ('{"user": {"$in": ["admin", "root"]}, "password": "password"}', 1),  # Variation: $in with multiple values
    ('{"username": {"$regex": "^{}$", "$options": "i"}}', 1),  # Variation: $regex with case-insensitive option
    ('{"user": {"$exists": true}, "password": {"$exists": true}}', 1),  # Variation: $exists with true
    ('{"user": {"$exists": false}, "password": {"$exists": false}}', 1),  # Variation: $exists with false
    ('{"name": {"$not": { "$type": "string" }}}', 1),  # Variation: $not with $type
    ('{"name": {"$mod": [5, 0]}}', 1),  # Variation: $mod with different divisor and remainder
    ('{"$where": "this.password.length > 10"}', 1),  # Variation: $where with length condition
    ('{"user": "test", "password": "pass"}', 0),  # Variation: benign query with different values
    ('{"name": "John Doe", "age": {}}', 0),  # Variation: benign query with different fields
    ('{"$or": [{"username": "test"}, {"isAdmin": false}]}', 0),  # Variation: benign query with different $or conditions
    ('{"user": "guest", "password": "guest"}', 0),  # Variation: benign query with different values
    ('{"type": "admin", "name": "admin", "roles": ["admin"], "password": "admin"}', 0),  # Variation: benign query with different roles
    ('{ "$and": [ {"username": "admin"}, {"isAdmin": true} ]}', 0),  # Variation: benign query with $and condition

    ('{"$nor": [{"username": "testuser"}, {"isAdmin": false}]}', 1),  # Variation: $nor condition with isAdmin
    ('{"user": {"$size": 3}, "password": "123456"}', 1),  # Variation: $size with a different array size
    ('{"$where": "this.age <= {}"}', 1),  # Variation: $where with <= condition
    ('{"name": {"$type": 2}}', 1),  # Variation: $type with a different BSON type (string)
    ('{"name": {"$type": 16}}', 1),  # Variation: $type with a different BSON type (int32)
    ('{"username": {"$regex": "^test", "$options": "i"}}', 1),  # Variation: $regex with case-insensitive option
    ('{"user": "guest", "password": {"$ne": "guest"}}', 1),  # Variation: $ne with a different value
    ('{"user": {"$regex": "^{}$"}, "password": "pass"}', 1),  # Variation: $regex with ^$ to match exactly
    ('{"username": {"$gt": null}}', 1),  # Variation: $gt with null
    ('{"username": {"$exists": false}, "password": {"$exists": true}}', 1),  # Variation: $exists with different combination
    ('{"name": {"$not": { "$type": 3 }}}', 1),  # Variation: $not with $type (Object)
    ('{"name": {"$not": { "$type": 18 }}}', 1),  # Variation: $not with $type (int64)
    ('{"$where": "this.password.length <= 5"}', 1),  # Variation: $where with length condition
    ('{"user": "test", "password": "password"}', 0),  # Variation: benign query with different values
    ('{"name": "Alice", "age": {}}', 0),  # Variation: benign query with different fields
    ('{"$or": [{"username": "user"}, {"isAdmin": true}]}', 0),  # Variation: benign query with different $or conditions
    ('{"user": "john", "password": "qwerty"}', 0),  # Variation: benign query with different values
    ('{"type": "user", "name": "user", "roles": ["user"], "password": "user"}', 0),  # Variation: benign query with different roles
    ('{ "$and": [ {"username": "test"}, {"isAdmin": false} ]}', 0),  # Variation: benign query with $and condition
    
    ('{"user": ["testuser", "admin"], "password": "password"}', 1),  # Variation: $in with an array of values
    ('{"username": {"$regex": "{}|admin", "$options": "i"}}', 1),  # Variation: $regex with | to match multiple patterns
    ('{"$and": [{"username": "test"}, {"isAdmin": false}] }', 1),  # Variation: $and condition with isAdmin
    ('{"user": {"$size": 1}, "password": "test"}', 1),  # Variation: $size with a different array size
    ('{"$where": "this.age > 20 && this.gender === \'male\'"}', 1),  # Variation: $where with multiple conditions
    ('{"name": {"$type": 4}}', 1),  # Variation: $type with a different BSON type (Array)
    ('{"name": {"$type": 17}}', 1),  # Variation: $type with a different BSON type (Timestamp)
    ('{"username": {"$regex": "^{}$"}, "password": {"$ne": "test"}}', 1),  # Variation: combined $regex and $ne conditions
    ('{"username": {"$gt": null}, "password": {"$eq": "password"}}', 1),  # Variation: combined $gt and $eq conditions
    ('{"name": {"$exists": true}, "password": {"$exists": false}}', 1),  # Variation: $exists with different combination
    ('{"name": {"$not": { "$type": 1 }}}', 1),  # Variation: $not with $type (Double)
    ('{"name": {"$not": { "$type": 19 }}}', 1),  # Variation: $not with $type (Array)
    ('{"$where": "this.password.length > 8 && this.name === \'admin\'"}', 1),  # Variation: $where with multiple conditions
    ('{"user": "admin", "password": "admin"}', 1),  # Variation: malicious query with common credentials
    ('{"user": "root", "password": "root"}', 1),  # Variation: malicious query with common credentials
    ('{"user": "admin", "password": "password123"}', 1),  # Variation: malicious query with weak password
    ('{"user": "testuser", "password": "testuser123"}', 1),  # Variation: malicious query with weak password
    ('{"user": "test", "password": "test"}', 0),  # Variation: benign query with different values
    ('{"name": "Bob", "age": {}}', 0),  # Variation: benign query with different fields
    ('{"$or": [{"username": "guest"}, {"isAdmin": false}]}', 0),  # Variation: benign query with different $or conditions
    ('{"user": "john", "password": "password123"}', 0),  # Variation: benign query with different values
    ('{"type": "user", "name": "user", "roles": ["user"], "password": "password"}', 0),  # Variation: benign query with different roles
    ('{ "$and": [ {"username": "user"}, {"isAdmin": false} ]}', 0),  # Variation: benign query with $and condition

    ('{"user": {"$elemMatch": {"$eq": "admin"}}, "password": {"$ne": "test"}}', 1),  # Variation: $elemMatch with $eq
    ('{"username": {"$regex": "{}.*admin.*", "$options": "i"}}', 1),  # Variation: $regex with .* before and after the pattern
    ('{"$and": [{"username": "test"}, {"isAdmin": true}] }', 1),  # Variation: $and condition with isAdmin
    ('{"user": {"$size": 2}, "password": "123456"}', 1),  # Variation: $size with a different array size
    ('{"$where": "this.age > 30 && this.gender === \'female\'"}', 1),  # Variation: $where with multiple conditions
    ('{"name": {"$type": 5}}', 1),  # Variation: $type with a different BSON type (Binary)
    ('{"name": {"$type": 20}}', 1),  # Variation: $type with a different BSON type (UUID)
    ('{"username": {"$regex": "^{}$", "$options": "i"}, "password": {"$ne": "admin"}}', 1),  # Variation: combined $regex and $ne conditions
    ('{"username": {"$gt": null}, "password": {"$eq": "admin"}}', 1),  # Variation: combined $gt and $eq conditions
    ('{"name": {"$exists": false}, "password": {"$exists": true}}', 1),  # Variation: $exists with different combination
    ('{"name": {"$not": { "$type": 6 }}}', 1),  # Variation: $not with $type (Undefined)
    ('{"name": {"$not": { "$type": 21 }}}', 1),  # Variation: $not with $type (Deprecated)
    ('{"$where": "this.password.length > 12 && this.name === \'root\'"}', 1),  # Variation: $where with multiple conditions
    ('{"user": "admin", "password": "administrator"}', 1),  # Variation: malicious query with common credentials
    ('{"user": "root", "password": "administrator"}', 1),  # Variation: malicious query with common credentials
    ('{"user": "admin", "password": "admin123"}', 1),  # Variation: malicious query with weak password
    ('{"user": "testuser", "password": "testuser2023"}', 1),  # Variation: malicious query with weak password
    ('{"user": "test", "password": "pass123"}', 0),  # Variation: benign query with different values
    ('{"name": "Jane", "age": {}}', 0),  # Variation: benign query with different fields
    ('{"$or": [{"username": "guest"}, {"isAdmin": true}]}', 0),  # Variation: benign query with different $or conditions
    ('{"user": "john", "password": "password2023"}', 0),  # Variation: benign query with different values
    ('{"type": "user", "name": "user", "roles": ["user"], "password": "secure123"}', 0),  # Variation: benign query with different roles
    ('{ "$and": [ {"username": "user"}, {"isAdmin": true} ]}', 0),  # Variation: benign query with $and condition
    ('{"type": "user", "name": "user", "roles": ["user"], "password": "password2023"}', 0),  # Variation: benign query with different roles and values
    ('{ "$and": [ {"username": "admin"}, {"isAdmin": false} ]}', 0),  # Variation: benign query with $and condition

    ('{"user": "malicious1", "password": "malicious1password"}', 1),
    ('{"username": {"$regex": "^malicious2"}}', 1),
    ('{"user": "test", "password": "test{}"}', 1),
    ('{"user": {"$in": ["admin", "root", "hacker"]}, "password": "password"}', 1),
    ('{"user": {"$size": 4}, "password": "123456"}', 1),
    ('{"$where": "this.age > 25 && this.gender === \'male\'"}', 1),
    ('{"name": {"$type": 22}}', 1),  # Variation: $type with a different BSON type (Symbol)
    ('{"name": {"$type": 23}}', 1),  # Variation: $type with a different BSON type (Scoped JavaScript Code)
    ('{"username": {"$regex": "^(malicious3|malicious4)\\\\d+"}}', 1),  # Variation: $regex with multiple patterns
    ('{"user": {"$eq": "admin"}, "password": "malicious5"}', 1),
    ('{"user": {"$regex": "^malicious6"}}', 1),
    ('{"$or": [{"isAdmin": true}, {"name": {"$regex": "^(malicious7|malicious8)"}}]}', 1),
    ('{"user": "malicious9", "password": {"$ne": "malicious9password"}}', 1),
    ('{"user": {"$in": ["admin", "root", "malicious10"]}, "password": "malicious10password"}', 1),

    ('{"user": {"$size": 10}, "password": {"$type": 200}}', 1),
    ('{"user": {"$regex": "^malicious23"}, "age": {"$exists": true}}', 1),
    ('{"user": "benign28", "name": {"$type": 2}}', 0),
    ('{"user": "benign27", "age": {"$exists": true}}', 0),
    ('db.categories.createIndex({"name": 1})', 0),
    ('db.reviews.remove({"user": "benign24"})', 0),
    ('db.orders.insert({"user": "user123", "product": "ProductA", "quantity": 2})', 0),
    ('db.customers.update({"user": "user123"}, {"$set": {"email": "new_email@example.com"}})', 0),
    

    ('{"user": "benign1", "password": "benign1password"}', 0),
    ('{"username": {"$regex": "^benign2"}}', 0),
    ('{"user": "test", "password": "test{}"}', 0),
    ('{"user": {"$in": ["guest", "user", "benign3"]}, "password": "password"}', 0),
    ('{"user": {"$size": 5}, "password": "123456"}', 0),
    ('{"$where": "this.age > 30 && this.gender === \'female\'"}', 0),
    ('{"name": {"$type": 24}}', 0),  # Variation: $type with a different BSON type (Deprecated (UUID))
    ('{"name": {"$type": 25}}', 0),  # Variation: $type with a different BSON type (Max key)
    ('{"username": {"$regex": "^(benign4|benign5)\\\\d+"}}', 0),  # Variation: $regex with multiple patterns
    ('{"user": {"$eq": "user"}, "password": "benign6"}', 0),
    ('{"user": {"$regex": "^benign7"}}', 0),
    ('{"$or": [{"isAdmin": false}, {"name": {"$regex": "^(benign8|benign9)"}}]}', 0),
    ('{"user": "benign10", "password": {"$ne": "benign10password"}}', 0),
    ('{"user": {"$in": ["guest", "user", "benign11"]}, "password": "benign11password"}', 0),

    ('{"user": "malicious11", "password": "malicious11password"}', 1),
    ('{"username": {"$regex": "^malicious12"}}', 1),
    ('{"user": "test", "password": "test123{}"}', 1),
    ('{"user": {"$in": ["admin", "root", "hacker", "malicious13"]}, "password": "password"}', 1),
    ('{"user": {"$size": 6}, "password": "123456"}', 1),
    ('{"$where": "this.age > 30 && this.gender === \'male\' && this.status === \'active\'"}', 1),
    ('{"name": {"$type": 26}}', 1),  # Variation: $type with a different BSON type (Min key)
    ('{"name": {"$type": 27}}', 1),  # Variation: $type with a different BSON type (Timestamp)
    ('{"username": {"$regex": "^(malicious14|malicious15)\\\\d+"}}', 1),  # Variation: $regex with multiple patterns
    ('{"user": {"$eq": "admin"}, "password": "malicious16"}', 1),
    ('{"user": {"$regex": "^malicious17"}}', 1),
    ('{"$or": [{"isAdmin": true}, {"name": {"$regex": "^(malicious18|malicious19)"}}]}', 1),
    ('{"user": "malicious20", "password": {"$ne": "malicious20password"}}', 1),
    ('{"user": {"$in": ["admin", "root", "malicious21", "hacker"]}, "password": "malicious21password"}', 1),


    ('{"user": "benign12", "password": "benign12password"}', 0),
    ('{"username": {"$regex": "^benign13"}}', 0),
    ('{"user": "test", "password": "test456{}"}', 0),
    ('{"user": {"$in": ["guest", "user", "benign14"]}, "password": "password"}', 0),
    ('{"user": {"$size": 7}, "password": "123456"}', 0),
    ('{"$where": "this.age > 35 && this.gender === \'female\' && this.status === \'active\'"}', 0),
    ('{"name": {"$type": 28}}', 0),  # Variation: $type with a different BSON type (Null)
    ('{"name": {"$type": 29}}', 0),  # Variation: $type with a different BSON type (Null)
    ('{"username": {"$regex": "^(benign15|benign16)\\\\d+"}}', 0),  # Variation: $regex with multiple patterns
    ('{"user": {"$eq": "user"}, "password": "benign17"}', 0),
    ('{"user": {"$regex": "^benign18"}}', 0),
    ('{"$or": [{"isAdmin": false}, {"name": {"$regex": "^(benign19|benign20)"}}]}', 0),
    ('{"user": "benign21", "password": {"$ne": "benign21password"}}', 0),
    ('{"user": {"$in": ["guest", "user", "benign22"]}, "password": "benign22password"}', 0),

    ('{"$where": "this.age > 35 && this.gender === \'female\' && this.isActive === true"}', 1),
    ('{"name": {"$type": 30}}', 1),  # Variation: $type with a different BSON type (Internal JavaScript Code)
    ('{"username": {"$regex": "^(malicious23|malicious24)\\\\d+"}, "isAdmin": false}', 1),
    ('{"user": {"$gt": "admin"}, "password": "maliciouspassword"}', 1),
    ('{"$or": [{"username": "malicious25"}, {"isAdmin": true}]}', 1),
    ('{"user": "malicious26", "password": {"$ne": "maliciouspassword"}}', 1),
    ('{"user": {"$in": ["admin", "root", "malicious27", "hacker", "test"]}, "password": "maliciouspassword"}', 1),


    ('{"$and": [{"username": "testuser"}, {"isAdmin": false}] }', 0),
    ('{"user": {"$size": 9}, "password": "benignpassword"}', 0),
    ('{"$where": "this.age > 40 && this.gender === \'male\' && this.isActive === true"}', 0),
    ('{"name": {"$type": 31}}', 0),  # Variation: $type with a different BSON type (Binary)
    ('{"username": {"$regex": "^(benign23|benign24)\\\\d+"}, "isAdmin": true}', 0),
    ('{"user": {"$gt": "user"}, "password": "benignpassword"}', 0),
    ('{"$or": [{"username": "benign25"}, {"isAdmin": false}]}', 0),
    ('{"user": "benign26", "password": {"$ne": "benignpassword"}}', 0),
    ('{"user": {"$in": ["guest", "user", "Superuser", "test"]}, "password": "password123"}', 0),
    ('{"user": {"$elemMatch": {"$eq": "user"}}, "password": {"$ne": "test"}}', 1),

    ('{"user": {"$elemMatch": {"$eq": "guest"}}, "password": {"$ne": "guest"}}', 1),
    ('{"user": {"$elemMatch": {"$eq": "admin"}}, "password": {"$ne": "admin"}}', 1),
    ('{"username": {"$regex": "^user"}, "isAdmin": true}', 1),
    ('{"username": {"$regex": "^guest"}, "isAdmin": true}', 1),
    ('{"username": {"$regex": "^admin"}, "isAdmin": true}', 1),
    ('{"user": {"$in": ["admin", "root", "hacker", "test", "guest"]}, "password": "password"}', 1),
    ('{"user": {"$in": ["user", "admin", "guest", "test", "hacker"]}, "password": "password"}', 1),
    ('{"user": {"$in": ["guest", "test", "user", "admin", "hacker"]}, "password": "password"}', 1),
    ('{"user": {"$size": 8}, "password": "123456"}', 1),
    ('{"user": {"$size": 10}, "password": "12345678"}', 1),
    ('{"$where": "this.age > 40 && this.gender === \'male\' && this.status === \'active\'"}', 1),
    ('{"$where": "this.age > 45 && this.gender === \'male\' && this.status === \'active\'"}', 1),
    ('{"name": {"$type": 32}}', 0),  # Variation: $type with a different BSON type (Timestamp)
    ('{"name": {"$type": 33}}', 0),  # Variation: $type with a different BSON type (32-bit Integer)
    ('{"name": {"$type": 34}}', 0),  
    ('{"username": {"$regex": "^(malicious28|malicious29)\\\\d+"}, "isAdmin": false}', 1),
    ('{"username": {"$regex": "^(malicious30|malicious31)\\\\d+"}, "isAdmin": false}', 1),
    ('{"user": {"$gt": "admin"}, "password": "maliciouspassword"}', 1),
    ('{"user": {"$gt": "user"}, "password": "maliciouspassword"}', 1),
    ('{"$or": [{"username": "malicious32"}, {"isAdmin": true}]}', 1),
    ('{"$or": [{"username": "malicious33"}, {"isAdmin": true}]}', 1),
    ('{"user": "malicious34", "password": {"$ne": "maliciouspassword"}}', 1),
    ('{"user": "malicious35", "password": {"$ne": "maliciouspassword"}}', 1),
    ('{"user": {"$in": ["admin", "root", "malicious36", "hacker", "test", "guest"]}, "password": "password123"}', 1),
    ('{"user": {"$in": ["user", "admin", "guest", "test", "hacker", "malicious37"]}, "password": "admin123"}', 1),

    # Additional unique commands for benign queries
    ('{"$and": [{"username": "guest"}, {"isAdmin": false}] }', 0),
    ('{"$and": [{"username": "test"}, {"isAdmin": true}] }', 0),
    ('{"user": {"$size": 11}, "password": "benignpassword"}', 0),
    ('{"user": {"$size": 12}, "password": "benignpassword"}', 0),
    ('{"$where": "this.age > 50 && this.gender === \'male\' && this.status === \'active\'"}', 0),
    ('{"$where": "this.age > 55 && this.gender === \'female\' && this.status === \'active\'"}', 0),
    ('{"username": {"$regex": "^(benign28|benign29)\\\\d+"}, "isAdmin": true}', 0),
    ('{"username": {"$regex": "^(benign30|benign31)\\\\d+"}, "isAdmin": true}', 0),
    ('{"user": {"$gt": "user"}, "password": "benignpassword"}', 0),
    ('{"user": {"$gt": "guest"}, "password": "benignpassword"}', 0),
    ('{"$or": [{"username": "admin1"}, {"isAdmin": false}]}', 0),
    ('{"$or": [{"username": "Admin123"}, {"isAdmin": false}]}', 0),
    ('{"user": "benign34", "password": {"$ne": "benignpassword"}}', 0),
    ('{"user": "benign35", "password": {"$ne": "benignpassword"}}', 0),
    ('{"user": {"$in": ["admin", "root", "benign36", "hacker", "test", "guest"]}, "password": "benignpassword"}', 0),
    ('{"user": {"$in": ["user", "admin", "guest", "test", "hacker", "Admin123"]}, "password": "benignpassword"}', 0),
]

In [26]:
#used to remove similar data present:


def is_similar(command1, command2, similarity_threshold=0.8):
    # Compare similarity of two commands using Jaccard similarity
    set1 = set(command1.split())
    set2 = set(command2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_similarity = intersection / union
    return jaccard_similarity >= similarity_threshold # returns a 1 or 0 

def clean_dataset(dataset, similarity_threshold=0.8):
    cleaned_commands = set()
    similar_commands = set()
    
    for command, label in dataset:
        is_duplicate = False
        is_similar_command = False
        
        # Check for exact duplicates
        if command in cleaned_commands:
            is_duplicate = True
        else:
            # Check for similar commands
            for cleaned_command in cleaned_commands:
                if is_similar(command, cleaned_command, similarity_threshold): # 0.9 is the similarity threshold above which two commands are considered similar and one of them is removed
                    similar_commands.add(command)
                    is_similar_command = True
                    break

        if not is_duplicate and not is_similar_command:
            cleaned_commands.add(command)

    cleaned_dataset = [(command, label) for command, label in dataset if command in cleaned_commands]
    return cleaned_dataset

# Assuming "dataset" contains the original dataset you provided


cleaned_dataset = clean_dataset(queries)
print(cleaned_dataset)


[('{"user": "{}", "password": "{}"}', 0), ('{"username": {"$regex": "^{}"}}', 1), ('{"username": {"$ne": "{}"}}', 1), ('{"username": {"$gt": "{}"}}', 1), ('{"user": "hacker", "password": {"$ne": ""}}', 1), ('{"user": req.query.user, "password": req.query.password}', 0), ('{"user": "hacker", "password": {"&ne": ""}}', 1), ('{"user": req.params[\'user\'], "password": req.params[\'password\']}', 0), ('{"user": request.GET[\'user\'], "password": request.GET[\'password\']}', 0), ('{"user": req.query.user_id, "password": req.query.user_pass}', 0), ('{"user": req.params[\'user_id\'], "password": req.params[\'user_pass\']}', 0), ('db.users.find({"user": "user123", "email": "user123@example.com"})', 0), ('db.users.find({"user": request.GET[\'user\'], "password": request.GET[\'password\']})', 0), ("db['users'].find({:user => req.params['user'], :password => req.params['password']})", 0), ('db.collection(\'users\').find({"user": "hacker", "password": {"&ne": ""}})', 1), ('}); db.dropDatabase(); d

In [28]:
#count of the benign and malicious commands:
label_1 = 0
label_0 = 0
for command, label in cleaned_dataset:
    if label == 1:
        label_1 += 1
    elif label == 0:
        label_0 += 1
overall = label_0+label_1
print(f"There are {label_1} malicious commands and {label_0} benign commands Concluding {overall} commands")


There are 221 malicious commands and 179 benign commands Concluding 400 commands


In [29]:
#characters dict
characters = ['"', "'", ';', '=', '{', '}', '(', ')', '[', ']', '$', '*', '|', ':', '^']

In [30]:
#randomly pick the commands in order to meet the number of benign and malicious quota

# Number of benign queries to generate
num_benign = 150

# Number of malicious queries to generate
num_malicious = 250

# Output file path
output_file = "dataset8.json"

# Generate benign queries
benign_samples = []
for _ in range(num_benign):
    query, label = random.choice(cleaned_dataset)
    if label == 0:  # For benign queries, use the JSON payload as it is
        payload = query
    else:  # For malicious queries, use the existing payload generation logic
        payload = random.choice(characters).join(query.split('"{}"'))
        payload += random.choice(characters)
    benign_samples.append((payload, label))

# Generate malicious queries
malicious_samples = []
for _ in range(num_malicious):
    query, label = random.choice(queries)
    payload = random.choice(characters).join(query.split('"{}"'))
    payload += random.choice(characters)
    malicious_samples.append((payload, label))

# Combine benign and malicious samples
samples = benign_samples + malicious_samples

# Shuffle the samples
random.shuffle(samples)

# Create a list of dictionaries for the dataset
dataset = []
for sample in samples:
    payload, label = sample
    data_point = {"text": payload, "label": label}
    dataset.append(data_point)

# Write dataset to the output file
with open(output_file, 'w') as f:
    json.dump(dataset, f, indent=2)

print(f"Dataset generated and saved to {output_file}")


Dataset generated and saved to dataset8.json


In [31]:
#command to just generate a json dataset consisting all the commands :
dataset = []
output_file = "dataset8abs.json"
for comd in cleaned_dataset:
    payload,label = comd
    data_point = {"text":payload, "label":label}
    dataset.append(data_point)

with open(output_file,'w')as f:
    json.dump(dataset, f, indent=2)

print(f"absolute dataset generated and saved to {output_file}")
    
    
    

absolute dataset generated and saved to dataset8abs.json
